<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Billing Management for Azure with Python

## Azure Actions Covered

* Listing billing accounts
* Listing billing profiles
* Listing and getting information on invoices
* Listing customers
* Listing available credit balances
* Listing account agreements with Microsoft

In this lecture, we'll learn how to use the Azure billing management library with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID). We'll add some new imports as well.

In [1]:
from azure.identity import AzureCliCredential
from azure.mgmt.billing import BillingManagementClient

from settings import AZURE_SUBSCRIPTION_ID

Let's instantiate our Azure credentials and the `BillingManagementClient()` object.

In [2]:
credential = AzureCliCredential()
billing_client = BillingManagementClient(credential, AZURE_SUBSCRIPTION_ID)

We can use the billing client to get information on the billing accounts in our subscription, such as the account status, account type, and agreement type.

In [5]:
print('Display Name - Account Status - Account Type - Agreement Type')
print('-------------------------------------------------------------')
for acct in billing_client.billing_accounts.list():
    print(f'{acct.display_name} - {acct.account_status} - {acct.account_type} - {acct.agreement_type}')

Display Name - Account Status - Account Type - Agreement Type
-------------------------------------------------------------
Benjamin Bigelow - Active - Individual - MicrosoftCustomerAgreement


There are some additional properties we can get, such as billing profiles, off the billing account objects. This will probably be empty if you're using an individual account for the course.

In [4]:
for acct in billing_client.billing_accounts.list():
    print(acct.billing_profiles)

{'additional_properties': {}, 'has_more_results': False, 'value': None}


Let's pick up the billing account name so that we can use it with other parts of the management client.

In [6]:
for acct in billing_client.billing_accounts.list():
    acct_name = acct.name

For any billing account, you can list the customers on the account and interact with them in certain ways. Since we don't have any customers, this will probably be an empty list.

In [27]:
for customer in billing_client.customers.list_by_billing_account(acct_name):
    print(customer)

We can also list out the invoices on our account. If you've recently started using Azure, an invoice may not have been generated yet.

In [8]:
print('Invoice # - Invoice Datetime - Profile Name - Amount Due')
print('--------------------------------------------------------')
for a in billing_client.invoices.list_by_billing_account(acct_name, '2023-01-01', '2023-06-01'):
    print(f'{a.name} - {a.invoice_date} - {a.billing_profile_display_name} - {a.amount_due.value} {a.amount_due.currency}')

Invoice # - Invoice Datetime - Profile Name - Amount Due
--------------------------------------------------------
G022901950 2023-05-09 05:14:36.264510+00:00 - Benjamin Bigelow 0.0 USD


If you know the ID/name for a particular invoice, you can get information on it via the invoices API.

In [9]:
invoice = billing_client.invoices.get(acct_name, 'G022901950')

Let's take a look at the billed and credited amount. Because this invoice was generated during a trial period on Azure, the billed and credited amounts will be the same.

In [12]:
print(f'Billed amount: {invoice.billed_amount} Credited amount: {invoice.credit_amount}')

Billed amount: {'additional_properties': {}, 'currency': 'USD', 'value': 28.13} Credited amount: {'additional_properties': {}, 'currency': 'USD', 'value': -28.13}


Let's take a look at the billing profiles available to us as well.

In [11]:
print('Billing Profile Name - Account Status')
print('------------------------------')
for profile in billing_client.billing_profiles.list_by_billing_account(acct_name):
    print(f'{profile.name} - {profile.status}')
    print(f'{profile.bill_to.first_name} {profile.bill_to.last_name} - {profile.bill_to.region.upper()}')

Billing Profile Name - Account Status
------------------------------
IIZN-6VR3-BG7-PGB - Active
Benjamin Bigelow - MT


If there are available credit balances to use, we can get them for a particular billing profile. Our credit balance will probably be returned as a `NoneType` object, since we don't have credits.

In [15]:
available_balance = billing_client.available_balances.get(acct_name, 'IIZN-6VR3-BG7-PGB')

In [21]:
available_balance.amount

Finally, you can see information about the billing accounts and their agreements with Microsoft.

In [23]:
for agreement in billing_client.agreements.list_by_billing_account(acct_name):
    print(f'{agreement.category} - {agreement.effective_date} - {agreement.expiration_date}')

MicrosoftCustomerAgreement - 2023-03-28 17:32:09.770242+00:00 - 9999-12-31 00:00:00+00:00
